In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold

from statistics import mean
import random
import csv
import time

import matplotlib.pyplot as plt
import seaborn as sn
plt.style.use('ggplot')
sn.set_style("whitegrid")
%matplotlib inline


from features import normaliseTrainData,normaliseTestData, time_domain_feature_gen, gen_rawData 
from features import load_data_paths, getInputVector, segment_df, concatenator, plot_confusion_matrix

Training segments shape:  (5398, 20, 12)
Testing segments shape:  (1078, 20, 12)
Input Training Vector shape:  (5398, 240)
Input Testing Vector shape:  (1078, 240)


## Constants 

In [2]:
TRAIN_FILEPATH = "./capstone_data/train/*.csv"
TEST_FILEPATH = "./capstone_data/test/*.csv"

DANCE_TO_NUM_MAP = {'dab': 0, 'jamesbond': 1, 'mermaid': 2}
RAW_COLS = ["acc_X", "acc_Y", "acc_Z", "gyro_X", "gyro_Y", "gyro_Z","subject", "trialNum", "dance"]

DANCE_MOVES = ["jamesbond", "dab", "mermaid"]
IN_FEATURES = 240
OUT_FEATURES = 3
HIDDEN_LAYER_1_NODES = 100
HIDDEN_LAYER_2_NODES = 60
K = 10   
EPOCHS = 100
TRAIN_BATCH_SIZE = 108
TEST_BATCH_SIZE = 72
LEARNING_RATE = 0.01
NUM_TO_DANCE_MAP = {0: 'dab', 1: 'jamesbond', 2: 'mermaid'}

## Load Dataset

In [3]:
# # test
# test_dic = generate_final_dataset(gen_rawData(load_data_paths(TEST_FILEPATH)))
# data_test, lbl_test = segment_df(concatenator(test_dic), "target")
# testing_X = getInputVector(data_test)
# print(f"testing_X input vector shape : {testing_X.shape}")
# print(f"labels for test shape: {lbl_test.shape}")

# # train
# train_dic = generate_final_dataset(gen_rawData(load_data_paths(TRAIN_FILEPATH)))
# data_train, lbl_train = segment_df(concatenator(train_dic), "target")
# training_X = getInputVector(data_train)
# print(f"training_X input vector shape : {training_X.shape}")
# print(f"labels for train shape: {lbl_train.shape}")

# testing

raw_test_df = concatenator(gen_rawData(load_data_paths(TEST_FILEPATH)))
raw_train_df = concatenator(gen_rawData(load_data_paths(TRAIN_FILEPATH)))
train_max = normaliseTrainData(raw_train_df)
normaliseTestData(raw_test_df, train_max)
feature_test_df = time_domain_feature_gen(raw_test_df)
feature_train_df = time_domain_feature_gen(raw_train_df)
feature_train_df["target"] = feature_train_df["dance"].map(DANCE_TO_NUM_MAP)
feature_test_df = time_domain_feature_gen(raw_test_df)
feature_test_df["target"] = feature_test_df["dance"].map(DANCE_TO_NUM_MAP)
segment_train_df = feature_train_df.drop(RAW_COLS, axis = 1)
segment_test_df = feature_test_df.drop(RAW_COLS, axis=1)
train_segs, lbl_train = segment_df(segment_train_df,"target")
test_segs, lbl_test = segment_df(segment_test_df, "target")
training_X = getInputVector(train_segs)
testing_X = getInputVector(test_segs)
print("Training segments shape: ", train_segs.shape)
print("Testing segments shape: ", test_segs.shape)
print("Input Training Vector shape: ", training_X.shape)
print("Input Testing Vector shape: ", testing_X.shape)

X_train = torch.FloatTensor(training_X)
X_test = torch.FloatTensor(testing_X)
y_train = torch.LongTensor(lbl_train)
y_test = torch.LongTensor(lbl_test)

Training segments shape:  (5398, 20, 12)
Testing segments shape:  (1078, 20, 12)
Input Training Vector shape:  (5398, 240)
Input Testing Vector shape:  (1078, 240)


## Multi-Layer Perceptron Model

In [4]:
class Model(nn.Module):
    def __init__(self, in_features, h1, h2, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

#### Instantiate Model class using default parameters

In [5]:
torch.manual_seed(32)
mlp = Model(in_features=IN_FEATURES, h1=HIDDEN_LAYER_1_NODES, h2=HIDDEN_LAYER_2_NODES, out_features=OUT_FEATURES)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=LEARNING_RATE)

#### 10-fold Stratified Cross Validation 

In [ ]:
skfcv = StratifiedKFold(n_splits=K, shuffle=True, random_state=1)
training_loss = {}
val_acc = []

# per fold 
for fold, (train_index, test_index) in enumerate(skfcv.split(X_train, y_train)):
    x_train_fold, x_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    train_combined = TensorDataset(x_train_fold, y_train_fold)
    test_combined = TensorDataset(x_test_fold, y_test_fold)
    trainloader = DataLoader(train_combined, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    testloader = DataLoader(test_combined, batch_size=TEST_BATCH_SIZE, shuffle=False)
    # per epoch
    for i in range(EPOCHS): 
        i+=1
        # per batch 
        losses = [] 
        val_correct_preds = 0
        count = 0 
        for batch_idx, (data, target) in enumerate(trainloader):
            mlp.train()
             
            # training 
            y_pred = mlp.forward(data)
            loss = criterion(y_pred, target)
            losses.append(loss)
            
            # backtracking 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
        
        # validating 
        with torch.no_grad():
            mlp.eval()
            for val_batch_idx, (val_data, val_target) in enumerate(testloader):
                y_out = mlp.forward(val_data)
                for row in range(y_out.shape[0]):
                    if y_out[row].argmax() == val_target[row]:
                        val_correct_preds += 1
                    count += 1
                    
    # per fold         
    with torch.no_grad():
        training_loss[fold] = np.array(losses).mean()
        print("-----------------------")
        print(f"fold: {fold} , training_loss: {training_loss[fold]}")
        print(f"fold: {fold}, {val_correct_preds} out of {count} = {100*val_correct_preds/count:.2f}% correct")
        print("-----------------------")
        val_acc.append(100*val_correct_preds/count)
        count = 0

print()
print("Done Training")
print("Max Validation Accuracy: ",  np.array(val_acc).max())

-----------------------
fold: 0 , training_loss: 3.972444062583236e-07
fold: 0, 532 out of 540 = 98.52% correct
-----------------------
-----------------------
fold: 1 , training_loss: 1.7308796032011742e-06
fold: 1, 538 out of 540 = 99.63% correct
-----------------------
-----------------------
fold: 2 , training_loss: 4.751834239868913e-06
fold: 2, 535 out of 540 = 99.07% correct
-----------------------
-----------------------
fold: 3 , training_loss: 2.669352397788316e-05
fold: 3, 539 out of 540 = 99.81% correct
-----------------------
-----------------------
fold: 4 , training_loss: 3.016773007402662e-06
fold: 4, 540 out of 540 = 100.00% correct
-----------------------
-----------------------
fold: 5 , training_loss: 9.74860267888289e-06
fold: 5, 538 out of 540 = 99.63% correct
-----------------------
-----------------------
fold: 6 , training_loss: 6.960767495911568e-05
fold: 6, 535 out of 540 = 99.07% correct
-----------------------
-----------------------
fold: 7 , training_loss

#### Mean Loss per fold 

In [ ]:
with torch.no_grad():
    y_axis  = training_loss.values()
    plt.plot(range(10), y_axis)
    plt.ylabel('Mean Loss per fold')
    plt.xlabel('Folds')

## Evaluation 

In [ ]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    start_time = time.time()
    y_val = mlp.forward(X_test)
    print("--- %s execution time in seconds ---" % (time.time() - start_time))
    loss = criterion(y_val, y_test)
    print(f'Loss with test set : {loss:.8f}')

In [ ]:
with torch.no_grad():
    # checking accuracy on test dataset 
    preds = []
    correct = 0
    for i,data in enumerate(X_test):
        y_val = mlp.forward(data)
        preds.append(y_val.argmax().item())
        if y_val.argmax().item() == y_test[i]:
            correct += 1
    print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')
    y_preds = torch.tensor(preds, dtype = torch.int64)
    stacked = torch.stack((y_test,y_preds),dim=1)

    # confusion matrix generation 
    cmt = torch.zeros(OUT_FEATURES,OUT_FEATURES, dtype=torch.int64)
    for p in stacked:
        tl, pl = p.tolist()
        cmt[tl, pl] = cmt[tl, pl] + 1  
    
    plt.figure(figsize=(12,12))
    plot_confusion_matrix(cmt, DANCE_MOVES)
    
    print("Classification Report for MLP :")
    print(classification_report(y_test, y_preds))

In [ ]:
# model weights 
with torch.no_grad():
    mlp_params = {}
    for name, param in mlp.named_parameters():
        mlp_params[name] = param.numpy().copy().tolist()
        
    print(mlp_params.keys())
    for key in mlp_params.keys(): 
        print(f"{key} : {len(mlp_params[key])} neurons in {key} layer")
        print(f" number of connections  : {len(mlp_params[key])} * {np.asarray(mlp_params[key][len(mlp_params[key]) -1]).size} = {len(mlp_params[key]) * np.asarray(mlp_params[key][len(mlp_params[key]) -1]).size } ")

    # save model weights 
    with open('mlp.csv', 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in mlp_params.items():
           writer.writerow([key, value])

## Save & Load Model

In [ ]:
torch.save(mlp.state_dict(), 'MLPW9.pt')
loaded_model = Model(in_features=IN_FEATURES,h1=HIDDEN_LAYER_1_NODES, h2=HIDDEN_LAYER_2_NODES, out_features=OUT_FEATURES) 
loaded_model.load_state_dict(torch.load('MLPW9.pt'))
loaded_model.eval()

In [ ]:
# loss when loaded model is run on test dataset 
with torch.no_grad():
    y_val = loaded_model.forward(X_test)
    loss = criterion(y_val, y_test)
    print(f'{loss:.8f}')

In [ ]:
# run model on a random sample row of the test dataset 
random.seed(69)
random_int = random.randint(0, len(testing_X))
random_input = torch.FloatTensor(testing_X[random_int])
print(random_input)

with torch.no_grad():
    print(loaded_model(random_input))
    print()
    print(loaded_model(random_input).max())
    print(loaded_model(random_input).argmax().item())
    print(f"Predicted Output: {NUM_TO_DANCE_MAP[loaded_model(random_input).argmax().item()]}")
    print(f"Actual Output: {NUM_TO_DANCE_MAP[y_test[random_int].item()]}")

In [ ]:
# save test dataset
with torch.no_grad(): 
    x = X_test.numpy().copy().tolist()
    np.savetxt("X_test.csv", x, delimiter=",")
    y = y_test.numpy().copy().tolist()
    np.savetxt("y_test.csv", y, delimiter=",")

In [ ]:
train_max